In [1]:
from pymongo import MongoClient
import pandas as pd
from pandas.io.json import json_normalize


In [2]:
def mongo(sector1,year):
    client = MongoClient('localhost', 27017)

    db = client['companies']
    comg = db.companies.find({"$and":[{'founded_year':{'$gte':year}},
                                      {'offices.latitude':{'$exists':True,'$ne':None}},
                                      {'offices.longitude':{'$exists':True,'$ne':None}},
                                      {'deadpooled_year': {'$eq':None}}, 
                             {'$or':
                              [{"category_code":sector1},
                               {"category_code":"games_video"},
                               {"category_code":"software"}]},
                             ]},
                             {'name':1, 'founded_year': 1, '_id': 0, 
                              'number_of_employees':1,'offices.latitude':1, 
                              'offices.longitude':1, 
                              'funding_rounds.raised_amount':1, 
                              'funding_rounds.raised_currency_code':1,
                              'category_code':1, 'total_money_raised':1})
    
    co=pd.DataFrame(comg)
    return co
    

- VISUALIZATION PROJECT Geospatial Business Intelligence (BI)
    * Make a geospartial analysis of the `companies` dataset
    * Things you know:
        - You have a software company with 50 employees
        - The company creates video games
        - Roles in your company: 20 developers, 20 Designers/Creatieves/UX/UI and 10 executives/managers
    * Do an analysis about placing the new company offices in the best environment based on the following criteria:
        - There should be software engineers working around
        - The surroundings must have a good ratio of big companies vs startups
        - Ensure you have in your surroundings companies that cover the interests of your team
        - Avoid old companies, prefer recently created ones







In [3]:
#pa ver la tabla
#client = MongoClient ('localhost', 27017)

#db = client.companies
#comg=db.companies.find({"$and":[{"category_code":"games_video"},{'founded_year':{'$gte':2003}},{'number_of_employees':{'$gte':5}},{'offices.latitude':{'$exists':True}},{'offices.longitude':{'$exists':True}},{'deadpooled_year':None}]},{'name':1, 'founded_year': 1, '_id': 0, 'number_of_employees':1,
                                 #'offices.latitude':1, 'offices.longitude':1, 'funding_rounds.raised_amount':1, 
                                 #'funding_rounds.raised_currency_code':1, 'category_code':1, 'total_money_raised':1})
#co=pd.DataFrame(comg)
#co.head()

In [4]:

#set(co)

In [5]:

def getlat(coor): 
    try: 
        return coor[0]['latitude']
    except:
        return None

   
def getlong(coor): 
    try: 
        return coor[0]['longitude']
    except:
        return None

   
def col_latlong(comp): 
    comp['longitude'] = comp['offices'].apply(getlong)
    comp['latitude'] = comp['offices'].apply(getlat)
    return comp

def getraised(funding): 
    try: 
        return funding[0]['raised_amount']
    except:
        return None
   
def getcurrency(funding): 
    try: 
        return funding[0]['raised_currency_code']
    except:
        return None

   
def col_funding(funding):   
    funding['raised_amount'] = funding['funding_rounds'].apply(getraised)
    funding['raised_currency_code'] = funding['funding_rounds'].apply(getcurrency)
    return funding

def col_drop(data):
    data = data.drop(['offices', 'funding_rounds'], axis=1)
    return data

In [6]:
#co.head()

In [7]:
companies = mongo("web",2008)
display(companies.head())
datalocation = col_latlong(companies)

datalocfund = col_funding(datalocation)
clean_companies = col_drop(datalocfund)
clean_companies.head()

,category_code,founded_year,funding_rounds,name,number_of_employees,offices,total_money_raised
0,software,2008,"[{'raised_amount': 950000, 'raised_currency_co...",GoingOn,40.0,"[{'latitude': 37.782263, 'longitude': -122.392...",$8.5M
1,web,2011,[],Mokitown,NaN,"[{'latitude': 37.09024, 'longitude': -95.712891}]",$0
2,games_video,2008,"[{'raised_amount': None, 'raised_currency_code...",YouBeQB,8.0,"[{'latitude': 47.597965, 'longitude': -122.151...",$0
3,games_video,2008,"[{'raised_amount': 4050000, 'raised_currency_c...",Crunchyroll,50.0,"[{'latitude': 37.781265, 'longitude': -122.393...",$4.8M
4,web,2013,"[{'raised_amount': 2000000, 'raised_currency_c...",Fixya,30.0,"[{'latitude': 37.566879, 'longitude': -122.323...",$8M


,category_code,founded_year,name,number_of_employees,total_money_raised,longitude,latitude,raised_amount,raised_currency_code
0,software,2008,GoingOn,40.0,$8.5M,-122.392142,37.782263,950000.0,USD
1,web,2011,Mokitown,NaN,$0,-95.712891,37.090240,NaN,None
2,games_video,2008,YouBeQB,8.0,$0,-122.151158,47.597965,NaN,None
3,games_video,2008,Crunchyroll,50.0,$4.8M,-122.393229,37.781265,4050000.0,USD
4,web,2013,Fixya,30.0,$8M,-122.323895,37.566879,2000000.0,USD


In [8]:
clean_companies.fillna(0)
clean_companies.head()

,category_code,founded_year,name,number_of_employees,total_money_raised,longitude,latitude,raised_amount,raised_currency_code
0,software,2008,GoingOn,40.0,$8.5M,-122.392142,37.782263,950000.0,USD
1,web,2011,Mokitown,NaN,$0,-95.712891,37.090240,NaN,None
2,games_video,2008,YouBeQB,8.0,$0,-122.151158,47.597965,NaN,None
3,games_video,2008,Crunchyroll,50.0,$4.8M,-122.393229,37.781265,4050000.0,USD
4,web,2013,Fixya,30.0,$8M,-122.323895,37.566879,2000000.0,USD


In [9]:
#clean_companies.to_json('clean_companies.json',orient='records')

In [10]:


#clean_companies = pd.DataFrame(db.restaurants.find({
    #"address.coord": {
     #"$near": {
       #"$geometry": {
          #"type": "Point" ,
          #"coordinates": [ nearLocation["lat"] , nearLocation["lng"] ]
       #},
       #"$maxDistance": 2000, # In meters
     #}
   #}
#}))
#clean_companies.head()

In [11]:
serie=[]
index=0
for i in clean_companies['longitude']:
    serie.append({"type": "Point","coordinates":[clean_companies['longitude'][index], clean_companies['latitude'][index]]})
    index+=1
serie

[{'type': 'Point', 'coordinates': [-122.392142, 37.782263]},
 {'type': 'Point', 'coordinates': [-95.712891, 37.09024]},
 {'type': 'Point', 'coordinates': [-122.151158, 47.597965]},
 {'type': 'Point', 'coordinates': [-122.393229, 37.781265]},
 {'type': 'Point', 'coordinates': [-122.323895, 37.566879]},
 {'type': 'Point', 'coordinates': [10.023246, 53.707739]},
 {'type': 'Point', 'coordinates': [-122.161674, 37.443058]},
 {'type': 'Point', 'coordinates': [-73.985433, 40.740231]},
 {'type': 'Point', 'coordinates': [-118.461884, 34.031764]},
 {'type': 'Point', 'coordinates': [-85.717393, 38.257035]},
 {'type': 'Point', 'coordinates': [-122.187925, 47.704875]},
 {'type': 'Point', 'coordinates': [151.039775, -34.054416]},
 {'type': 'Point', 'coordinates': [51.5082, 25.2948]},
 {'type': 'Point', 'coordinates': [-122.404488, 37.790032]},
 {'type': 'Point', 'coordinates': [3.165225, 50.694712]},
 {'type': 'Point', 'coordinates': [-122.161523, 37.446823]},
 {'type': 'Point', 'coordinates': [-80.

In [12]:
clean_companies['geo']=serie
#clean_companies.dropna( inplace=True )
clean_companies.head()

,category_code,founded_year,name,number_of_employees,total_money_raised,longitude,latitude,raised_amount,raised_currency_code,geo
0,software,2008,GoingOn,40.0,$8.5M,-122.392142,37.782263,950000.0,USD,"{'type': 'Point', 'coordinates': [-122.392142,..."
1,web,2011,Mokitown,NaN,$0,-95.712891,37.090240,NaN,None,"{'type': 'Point', 'coordinates': [-95.712891, ..."
2,games_video,2008,YouBeQB,8.0,$0,-122.151158,47.597965,NaN,None,"{'type': 'Point', 'coordinates': [-122.151158,..."
3,games_video,2008,Crunchyroll,50.0,$4.8M,-122.393229,37.781265,4050000.0,USD,"{'type': 'Point', 'coordinates': [-122.393229,..."
4,web,2013,Fixya,30.0,$8M,-122.323895,37.566879,2000000.0,USD,"{'type': 'Point', 'coordinates': [-122.323895,..."


In [13]:
clean_companies.to_json('clean_companiess.json',orient='records' ,lines=True)